## PL Trainer for training model

<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
#| code-fold: show
#| code-summary: "Exported source"
import lightning as pl
import torch
import torch.nn as nn
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from transformers import AdamW, get_linear_schedule_with_warmup

from en_grammar_checker.models import EnDeBertaClassifier

In [ ]:
from en_grammar_checker.config import Config
from en_grammar_checker.datasets import get_train_data_loader

In [1]:
#| echo: false
#| output: asis
show_doc(MyLightningClassifierModel)

---

[source](https://github.com/rohitMalhotra07/en_grammar_checker/blob/main/en_grammar_checker/trainer.py#L20){target="_blank" style="float:right; font-size:smaller"}

### MyLightningClassifierModel

>      MyLightningClassifierModel (cnfg)

*Hooks to be used in LightningModule.*

In [ ]:
#| code-fold: show
#| code-summary: "Exported source"
# define the LightningModule
class MyLightningClassifierModel(pl.LightningModule):
    def __init__(self, cnfg):
        super().__init__()
        self.save_hyperparameters()

        self.model = EnDeBertaClassifier(cnfg)

        self.learning_rate = cnfg.learning_rate
        self.eps = cnfg.eps
        self.weight_decay = cnfg.weight_decay
        self.betas = cnfg.betas
        self.warmup_prop = cnfg.warmup_prop

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, x2, y = batch
        # y = y.view(-1)
        pred = self.model(input_ids=x, attention_mask=x2)
        loss = loss_fct(pred, y)

        # Logging to TensorBoard (if installed) by default
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        loss, pred, y, _f1, _precision, _recall = self._shared_eval_step(
            batch, batch_idx
        )
        # metrics = {
        #     "val_loss": loss,
        #     "val_f1": _f1,
        #     "val_precision": _precision,
        #     "val_recall": _recall,
        # }
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        self.log(
            "val_f1", _f1, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )

        self.log(
            "val_precision",
            _precision,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        self.log(
            "val_recall",
            _recall,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        self.validation_step_y_hat.append(pred)
        self.validation_step_y.append(y)

        return loss

    def on_validation_epoch_start(self):
        self.validation_step_y_hat = []  # free memory
        self.validation_step_y = []

    def on_validation_epoch_end(self):
        all_preds = torch.cat(self.validation_step_y_hat)
        all_y = torch.cat(self.validation_step_y)

        all_preds = torch.argmax(all_preds, axis=1).flatten().detach().cpu()
        all_y = all_y.flatten().detach().cpu()

        # do something with all preds
        _f1 = f1_score(all_preds, all_y, average="macro")
        _precision = precision_score(all_preds, all_y, average="macro")
        _recall = recall_score(all_preds, all_y, average="macro")
        # metrics = {
        #     "val_f1_full": _f1,
        #     "val_precision_full": _precision,
        #     "val_recall_full": _recall,
        # }
        # self.log_dict(metrics)

        print(
            f"\t Epoch: {self.current_epoch}, Val F1: {_f1}, Val Precision: {_precision}, Val Recall {_recall}"
        )

    def _shared_eval_step(self, batch, batch_idx):
        x, x2, y = batch
        # y = y.reshape(-1, 1)
        pred = self.model(input_ids=x, attention_mask=x2)
        loss = loss_fct(pred, y)

        _f1 = f1_score(
            torch.argmax(pred, axis=1).cpu().numpy(), y.cpu().numpy(), average="macro"
        )
        _precision = precision_score(
            torch.argmax(pred, axis=1).cpu().numpy(), y.cpu().numpy(), average="macro"
        )

        _recall = recall_score(
            torch.argmax(pred, axis=1).cpu().numpy(), y.cpu().numpy(), average="macro"
        )
        return loss, pred, y, _f1, _precision, _recall

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, x2 = batch
        pred = self.model(x)
        return pred

    def configure_optimizers(self):
        # return optimizer
        optimizer = AdamW(
            self.model.parameters(),
            lr=self.learning_rate,
            eps=self.eps,
            weight_decay=self.weight_decay,
            betas=self.betas,
        )
        # get_linear_schedule_with_warmup(
        #     optimizer,
        #     num_warmup_steps=len(self.train_dataloader)
        #     * self.num_epochs
        #     * self.warmup_prop,
        #     num_training_steps=len(self.train_dataloader) * self.num_epochs,
        # )

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            patience=5,
            factor=0.1,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "train_loss",
        }